In [232]:
"""
Скрипт для подсчета общих данных по ЦК
"""

'\nСкрипт для подсчета общих данных по ЦК\n'

In [233]:
import pandas as pd
import numpy as np
import openpyxl
import os
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import copy

In [234]:
def check_horizont_all_sum_error(df:pd.DataFrame,tup_exluded_cols:tuple,name_itog_cols,name_file):
    """
    Функция для проверки горизонтальных сумм по всей строке
    сумма в колонке 05 должна быть равна сумме всех колонок за исключением 07 и 15
    """
    # датафрейм для ощибок по горизонтали
    hor_error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки'])
    
    # получаем список колонок
    all_sum_cols =list(df)
    # удаляем колонки
    for name_cols in tup_exluded_cols:
        all_sum_cols.remove(name_cols)
    # удаляем итоговую колонку
    all_sum_cols.remove(name_itog_cols)

    # получаем сумму колонок за вычетом исключаемых и итоговой колонки
    df['Сумма'] = df[all_sum_cols].sum(axis=1)
    # Проводим проверку
    df['Результат'] = df[name_itog_cols] == df['Сумма']
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
    # получаем датафрейм с ошибками и извлекаем индекс
    df = df[df['Результат'] == 'Неправильно'].reset_index()
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist()  # делаем список
    finish_lst_index = list(map(lambda x: x + 1, raw_lst_index))
    finish_lst_index = list(map(lambda x: f'Строка 0{str(x)}', finish_lst_index))
    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = f'Не выполняется условие: гр. {name_itog_cols} = сумма остальных гр. за искл.{tup_exluded_cols}'
    return temp_error_df
    
    

    
    
    

In [235]:
def check_horizont_chosen_sum_error(df:pd.DataFrame,tup_checked_cols:list,name_itog_cols,name_file):
    """
    Функция для проверки равенства одиночных или небольших групп колонок
    tup_checked_cols колонки сумму которых нужно сравнить с name_itog_cols чтобы она не превышала это значение
    """
    # Считаем проверяемые колонки
    df['Сумма'] = df[tup_checked_cols].sum(axis=1)
     # Проводим проверку
    df['Результат'] = df[name_itog_cols] >= df['Сумма']
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
    # получаем датафрейм с ошибками и извлекаем индекс
    df = df[df['Результат'] == 'Неправильно'].reset_index()
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist()  # делаем список
    finish_lst_index = list(map(lambda x: x + 1, raw_lst_index))
    finish_lst_index = list(map(lambda x: f'Строка 0{str(x)}', finish_lst_index))
    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = f'Не выполняется условие: гр. {name_itog_cols} >= сумма {tup_checked_cols}'
    return temp_error_df
    
    

In [236]:
def check_error_ck(df:pd.DataFrame,name_file):
        # создаем датафрейм для регистрации ошибок
    error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки'])
    
    # проводим горизонтальные проверки
    # проверка на общую сумму
    first_error_ck_df  = check_horizont_all_sum_error(df.copy(),('07','15'),'05',name_file) 
    error_df = pd.concat([error_df,first_error_ck_df],axis=0,ignore_index=True)
    # проверяем небольшие группы или одиночные колонки
    second_error_ck_df  = check_horizont_chosen_sum_error(df.copy(),['07'],'06',name_file) 
    error_df = pd.concat([error_df,second_error_ck_df],axis=0,ignore_index=True)
    
    #проверяем колонки 14 и 15
    third_error_ck_df  = check_horizont_chosen_sum_error(df.copy(),['15'],'14',name_file) 
    error_df = pd.concat([error_df,third_error_ck_df],axis=0,ignore_index=True)
    
    
    
    
    return error_df

 
        
    
    


In [237]:
path_to_data_ck = 'data/Отчет ЦК'


# создаем базовый датафрейм заполненный нулями
base_df = pd.DataFrame(np.zeros((5,27)))
base_df=base_df.applymap(int) # приводим его к инту
base_df.columns = ['05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24',
                   '25','26','27','28','29','30','31']


# создаем датафрейм для регистрации ошибок
base_error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ])

for file in os.listdir(path_to_data_ck):
    if not file.startswith('~$') and file.endswith('.xlsx'):
        name_file = file.split('.xlsx')[0]
        temp_df_ck = pd.read_excel(f'{path_to_data_ck}/{file}',skiprows=5,nrows=5)
        # Создаем копию датафрейма только с числовыми колонками
        temp_df_int = temp_df_ck.iloc[:,2:29].copy()
        # заменяем пустые ячейки нулями

        temp_df_int.fillna(0,inplace=True)
        temp_df_int = temp_df_int.applymap(int)

        # проверяем на ошибки
        temp_error_df = check_error_ck(temp_df_int.copy(),name_file)
        base_error_df = pd.concat([base_error_df,temp_error_df],axis=0,ignore_index=True)
        # проверяем размер датафрейма с ошибками, если их нет то добавляем в результат.
        
        



In [238]:
base_error_df.to_excel('проверка.xlsx',index=False)

In [239]:
base_df + temp_df_int

,05,06,07,08,09,10,11,12,13,14,...,22,23,24,25,26,27,28,29,30,31
0,11,5,3,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,3,3,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
